In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

root = 'https://www.privateproperty.co.za'
website = f'{root}/for-sale/western-cape/cape-town/cape-town-city-bowl/oranjezicht/697'
result = requests.get(website)
content = result.text
soup = BeautifulSoup(content, 'lxml')

box =soup.find('div', class_='resultsItemsContainer')
box.find_all('a', href=True)

for link in box.find_all('a', href=True):
    link['href']

links = [link['href'] for link in box.find_all('a', href=True)]
links = [item for item in links if item.count('developments') < 1]
print(links)


In [ ]:
titles=[] #Listing name on private property
prices=[] #List to store price of each listing
addresses=[] #List to store price of each listing
features=[] #List of features for each listing

pages = np.arange(1, 8, 1)
#iterates through each page and saves data on each listing
for page in pages:
    page="https://www.privateproperty.co.za/for-sale/western-cape/cape-town/cape-town-city-bowl/oranjezicht/697?page=" + str(page) 
    for link in links:
        result = requests.get(f'{root}/{link}')
        content = result.text
        soup = BeautifulSoup(content, 'lxml')

        price = soup.find('span', class_='detailsPrice').get_text()
        prices.append(price)

        box =soup.find('div', class_='titleContainer')
        title = box.find('h1').get_text()
        titles.append(title)

        address =soup.find('div', class_='address')
        if address == (None):
            addresses.append('No Address')
        else:
            addresses.append(address.text)

        feature = soup.find('div', class_='mainFeatures').get_text()
        features.append(feature)
        

In [ ]:
df = pd.DataFrame({'Listing':titles,'Price':prices, 'Address':addresses, 'Features':features})
print(df)

In [ ]:
temp_df = df["Features"].str.split("\n", n = 4, expand = True)
Floor_space = temp_df[1]
Floor_space = Floor_space.str.replace(r'\D', '')
temp_df[1] = Floor_space

df_final = pd.concat([df, temp_df[1]], axis=1)
df_final = df_final.drop("Features", axis = 'columns')
df_final = df_final.rename (columns = {1: 'Floor_space_m²'})
print(df_final)

df_final.to_csv("housing.csv")
